## LR

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from collections import namedtuple

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, roc_curve

##### 获取数据

In [9]:
def get_criteo_data():
    """ 读取criteo数据集 """
    train_path = '../data/criteo/train.txt'
    test_path = '../data/criteo/test.txt'

    dense_column_names = ['I' + str(i) for i in range(1, 14)]
    sparse_column_names = ['C' + str(i) for i in range(14, 40)]
    column_names = ['label'] + dense_column_names + sparse_column_names

    train_df = pd.read_csv(train_path, names=column_names, sep='\t')
    test_df = pd.read_csv(test_path, names=column_names, sep='\t')
    return train_df, test_df, dense_column_names, sparse_column_names

train_df, test_df, dense_column_names, sparse_column_names = get_criteo_data()
print('train.shape: {}, test.shape: {}'.format(train_df.shape, test_df.shape))

train.shape: (1000000, 40), test.shape: (1000000, 40)


##### 数据预处理

In [10]:
data = pd.concat([train_df, test_df], axis=0)

def data_processing(df, dense_column_names, sparse_column_names):
    """ 数据预处理 """
    df[dense_column_names] = df[dense_column_names].fillna(0.0)
    for f in dense_column_names:
        df[f] = df[f].apply(lambda x: np.log(x+1) if x > -1 else -1)
    
    df[sparse_column_names] = df[sparse_column_names].fillna("-1")
    for f in sparse_column_names:
        lbe = LabelEncoder()
        df[f] = lbe.fit_transform(df[f])
    return df[dense_column_names + sparse_column_names]

df = data_processing(data, dense_column_names, sparse_column_names)
df['label'] = data['label']

train_df, test_df = df.iloc[0:train.shape[0]], df.iloc[train.shape[0]:]
print('train.shape:{}, test.shape:{}'.format(train_df.shape, test_df.shape))

train.shape:(1000000, 40), test.shape:(1000000, 40)


In [11]:
train_df.head()

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,C31,C32,C33,C34,C35,C36,C37,C38,C39,label
0,0.693147,0.693147,1.791759,0.000000,7.232010,1.609438,2.772589,1.098612,5.204007,0.693147,...,4053,237,3,13436,0,3,46295,68,25504,0
1,1.098612,0.000000,3.806662,0.693147,4.634729,2.197225,1.098612,1.098612,1.609438,0.693147,...,2904,237,1,167950,0,3,15898,68,19236,0
2,1.098612,0.000000,0.693147,2.708050,6.643790,4.499810,1.609438,1.098612,5.505332,0.693147,...,850,0,0,397303,11,3,13834,0,0,0
3,0.000000,6.795706,0.000000,0.000000,8.387768,0.000000,0.000000,0.000000,0.000000,0.000000,...,1913,0,0,185717,0,3,33810,0,0,0
4,1.386294,-1.000000,0.000000,0.000000,1.098612,0.000000,1.386294,0.000000,0.000000,0.693147,...,639,0,0,58474,0,2,41894,0,0,0


##### 模型构建

In [15]:
SparseFeature = namedtuple('SparseFeature', ['name', 'vocabulary_size', 'embedding_size'])
DenseFeature = namedtuple('DenseFeature', ['name', 'dimension'])

feature_columns = [SparseFeature(f, vocabulary_size=df[f].nunique(), embedding_size=4) for f in sparse_column_names] + \
[DenseFeature(f, 1) for f in dense_column_names]

In [16]:
def build_input_layers(feature_columns):
    """ 构建输入层 """
    dense_input_dict, sparse_input_dict = {}, {}
    for f in feature_columns:
        if isinstance(f, DenseFeature):
            dense_input_dict[f.name] = Input(shape=(f.dimension, ), name=f.name)
        elif isinstance(f, SparseFeature):
            sparse_input_dict[f.name] = Input(shape=(1, ), name=f.name)
    return dense_input_dict, sparse_input_dict

def build_embedding_layers(feature_columns, is_linear):
    """ 构建embedding层 """
    embedding_layer_dict = {}
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    if is_linear:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size + 1, 1, name='1d_emb_' + f.name)
    else:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size + 1, f.embedding_size, name='kd_emb_' + f.name)
    return embedding_layer_dict
    
def get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns):
    """ 线性回归 """
    concat_dense_inputs = Concatenate(axis=1)(list(dense_input_dict.values()))
    dense_logits_output = Dense(1)(concat_dense_inputs)
    
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    embedding_layer_dict = build_embedding_layers(sparse_feature_columns, is_linear=True)
    sparse_1d_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = Flatten()(embedding_layer_dict[f.name](_input))
        sparse_1d_embed_list.append(_embed)
    if len(sparse_1d_embed_list) > 0:
        sparse_logits_output = Add()(sparse_1d_embed_list)
        linear_logits = Add()([dense_logits_output, sparse_logits_output])
    else:
        linear_logits = dense_logits_output
    return linear_logits

def model_metric(prob, label, thr=0.5):
    """ 模型评估 """
    # AUC
    fpr, tpr, threshold = metrics.roc_curve(label, prob)
    auc = metrics.auc(fpr, tpr)
    score = metrics.accuracy_score(label, prob > thr)
    # LogLoss
    logloss = log_loss(label, prob)
    print('模型准确率:{}, AUC得分:{}, LogLoss:{}'.format(score, auc, logloss))
    print(classification_report(label, prob > thr, digits=2))
    print('==========================================================')

def LR(feature_columns, seed=1024, l2_reg=1e-5, task='binary'):
    """ Instantiates LR architecture
    :param feature_columns 
    :param seed
    :param l2_reg L2regularization
    :return: A kears model instance
    """
    dense_input_dict, sparse_input_dict = build_input_layers(feature_columns)
    input_list = list(dense_input_dict.values()) + list(sparse_input_dict.values())
    
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns)
    output_layer = Activation("sigmoid")(linear_logits)
    model = Model(input_list, output_layer)
    return model

model = LR(feature_columns)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C17 (InputLayer)                [(None, 1)]          0                                            
______________________________________________________________________________________________

In [20]:
train_input = {f: train_df[f] for f in dense_column_names + sparse_column_names}
test_input = {f: test_df[f] for f in dense_column_names + sparse_column_names}

my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=2, mode='auto')
]
model.compile('adam',
              loss='binary_crossentropy',
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])
model.fit(train_input,
          train_df['label'].values,
          batch_size=1024,
          epochs=100,
          validation_split=0.2,
          callbacks=my_callbacks)

Epoch 1/100
782/782 [==============================] - 14s 14ms/step - loss: 0.7199 - binary_crossentropy: 0.7199 - auc: 0.5491 - val_loss: 0.5088 - val_binary_crossentropy: 0.5088 - val_auc: 0.7136
Epoch 2/100
782/782 [==============================] - 11s 14ms/step - loss: 0.4800 - binary_crossentropy: 0.4800 - auc: 0.7630 - val_loss: 0.4769 - val_binary_crossentropy: 0.4769 - val_auc: 0.7616
Epoch 3/100
782/782 [==============================] - 11s 14ms/step - loss: 0.4478 - binary_crossentropy: 0.4478 - auc: 0.8058 - val_loss: 0.4671 - val_binary_crossentropy: 0.4671 - val_auc: 0.7749
Epoch 4/100
782/782 [==============================] - 11s 15ms/step - loss: 0.4293 - binary_crossentropy: 0.4293 - auc: 0.8281 - val_loss: 0.4645 - val_binary_crossentropy: 0.4645 - val_auc: 0.7784
Epoch 5/100
782/782 [==============================] - 11s 14ms/step - loss: 0.4163 - binary_crossentropy: 0.4163 - auc: 0.8420 - val_loss: 0.4641 - val_binary_crossentropy: 0.4641 - val_auc: 0.7788
Epoch

In [21]:
# 模型预测与评估
result = model.predict(test_input)
model_metric(np.array([i[0] for i in result]), test_df['label'].values)

模型准确率:0.777775, AUC得分:0.7608384236148322, LogLoss:0.48295328287262196
              precision    recall  f1-score   support

           0       0.81      0.91      0.86    751819
           1       0.58      0.37      0.45    248181

    accuracy                           0.78   1000000
   macro avg       0.70      0.64      0.66   1000000
weighted avg       0.76      0.78      0.76   1000000

